<!--NAVIGATION-->
< [Catchment Delineation](CatchmentDelineation.ipynb)  | [Contents](ContentsCatchmentDelineation.ipynb)>

# 8. Automatisation de la délimitation des cours d'eau et des bassins versants
Nous pouvons également automatiser la procédure de délimitation des cours d'eau et du bassin versant d'un exutoire donné en utilisant le code des sections précédentes. Nous ne pouvons évidemment pas automatiser la calibration de la délimitation des cours d'eau.

Essayez d'abord de combiner tout le code des sections précédentes par vous-même. Ecrivez-le dans des fonctions pour le rendre plus lisible. Vous pouvez utiliser l'exemple de code ci-dessous pour vous aider.

In [ ]:
import os, glob
from pcraster import *
from osgeo import gdal, gdalconst

def mosaic(inputpattern,outputmosaic):
    InputFiles = glob.glob(inputpattern)
    mosaic = gdal.BuildVRT(outputmosaic,InputFiles)
    mosaic.FlushCache()
    
def reprojectAndClip(inputraster,outputraster,projection,shapefile,resolution):
    options = gdal.WarpOptions(cutlineDSName=shapefile,
                           cropToCutline=True,
                           format='GTIFF',
                           dstSRS=projection,
                           xRes=resolution,
                           yRes=resolution)
    outimage=gdal.Warp(srcDSOrSrcDSTab=inputraster,
                           destNameOrDestDS=outputraster,
                           options=options)

def ConvertToPCRaster(src_filename,dst_filename,ot,VS):
    #Ouvrir un ensemble de données disponible
    src_ds = gdal.Open( src_filename )
    
    #GDAL Translate
    dst_ds = gdal.Translate(dst_filename, src_ds, format='PCRaster', outputType=ot, metadataOptions=VS)
    
    #Fermez correctement les ensembles de données pour les transférer sur le disque
    dst_ds = None
    src_ds = None
    
def CalculateFlowDirection(DEMFile):
    DEM = readmap(DEMFile)
    FlowDirectionMap = lddcreate(DEM,1e31,1e31,1e31,1e31)
    return FlowDirectionMap

def StreamDelineation(FlowDirectionMap,Threshold):
    StrahlerOrders = streamorder(FlowDirectionMap)
    Stream = ifthen(StrahlerOrders >= Threshold, boolean(1))
    return Stream
    
def col2map(x,y,clone):
    with open('location.txt', 'w') as f:
        f.write(str(x) + ' ' + str(y) + ' 1')
    cmd = 'col2map -N location.txt location.map --clone {0}'.format(clone)
    os.system(cmd)
    Map = readmap('location.map')
    return Map
    

# Définir les entrées et les paramètres
# Notez que les sorties (résultats)des exécutions précédentes doivent être supprimées pour éviter les erreurs
os.chdir('../PCRasterCatchmentDelineation/')
TileExtension = '*.tif'
MosaicOutput = 'mosaic.vrt'
BoundaryPolygon = 'boundingbox.shp'
OutputProjection = 'EPSG:32632'
OutputSpatialResolution = 30.0
DEMSubsetOutput = 'DEMsubset.tif'
PCRasterDEMOutput = 'dem.map'
FlowDirectionOutput = 'flowdir.map'
StrahlerOrderThreshold = 8
OutletX = 288880.648
OutletY = 5675880.258
clone = PCRasterDEMOutput


# Appliquer le cheminement (protocole) de la délimitation des cours d'eau et des bassins versants
print('Creating mosaic...')
mosaic(TileExtension,MosaicOutput)
print('Done!')

print('Reprojecting and clipping...')
reprojectAndClip(MosaicOutput,DEMSubsetOutput,OutputProjection,BoundaryPolygon,OutputSpatialResolution)
print('Done')

print('Converting to PCRaster format...')
ConvertToPCRaster(DEMSubsetOutput,PCRasterDEMOutput,gdalconst.GDT_Float32,"VS_SCALAR")
print('Done!')

print('Calculating flow direction...')
setclone(clone)
FlowDirection = CalculateFlowDirection(PCRasterDEMOutput)
print('Done!')

print('Delineating channels...')
River = StreamDelineation(FlowDirection,StrahlerOrderThreshold)
print('Done')

print('Delineating the catchment...')
Outlet = col2map(OutletX,OutletY,clone)
CatchmentArea = catchment(FlowDirection,Outlet)
print('Done')

#Visualisez ce que vous souhaitez
aguila(CatchmentArea)
aguila(FlowDirection)
aguila(River)

#Rapportez ce que vous souhaitez
report(CatchmentArea,'catchment.map')
report(FlowDirection,'flowdir.map')
report(River,'channels.map')

<!--NAVIGATION-->
< [Catchment Delineation](CatchmentDelineation.ipynb)  | [Contents](ContentsCatchmentDelineation.ipynb)>